#### Student Name : Ayush K. Rai
#### Program : MSc in Artificial Intelligence


**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd, Adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Flatten

Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

> In the method `Agent.act(state, train=True)` , the agent attempts to decide which action to take. The method takes the current state as the input represented by (`state`) and predicts the action to perform from that state, which is also the output of the function.

> However the best action to perform given a state is something that an agent needs to learn through a learning process. In the begining the agent doesn't have any knowledge about performing the correct action. Therefore the agent has to experiment new behaviours based on trial and error to find the action which yields the greatest expected cumulative discounted reward (or expected return $R(\pi)$).

> This trial and error based behaviour is otherwise called exploration and exploitation policy. One such type of policy is $\varepsilon$-greedy (`Agent.epsilon`) exploration policy. With some probibility $\varepsilon$ (also called exploration rate), the agent explores (i.e takes an action at random) and otherwise, the agent exploits the policy that it has learnt.

> In practice the agent needs to explore more in the initial stages and slowly descrease the value of $\varepsilon$ and therefore take more actions based on the knowledge and experience acquired till then with a higher probability.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=10 # set small when debugging
epochs_test=10 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

*Position*  
This matrix is used to define the rat position and allowed moves. 
- It is filled with `0` by default.
- On every border, there is thickness of two cell with value `-1`, representing the cells where the rat cannot go. That's why +4 was added to the grid size, allowing space for these borders.


- Only one cell is set to `1`, representing the actual positon of the rat.

*Board*  
This matrix is used to define the environment.
- It is also initialized with a size increased by 4, to fit with the `positon matrix`.
- It contains `rewards` for the rat. `-1` for poison, `0.5` for cheese.
- When the rat has already explored a cell, we set the cell to `0`

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        a = np.random.randint(0, self.n_action)
        return a

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
 
    for e in range(epochs):
        
        state = env.reset()
        
        # This assumes that the games will end
        game_over = False
        
        #Initialization of the Variables
        win = 0
        lose = 0

        while not game_over:
            
            action = agent.learned_act(state)
            
            #print(action)
            
            prev_state= state
            state,reward,game_over = env.act(action)
            
            if reward >0:
                win = win +reward 
            if reward < 0:
                lose = lose-reward
                
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 10.5/10.0. Average score (0.5)
Win/lose count 7.0/15.0. Average score (-3.75)
Win/lose count 11.0/15.0. Average score (-3.8333333333333335)
Win/lose count 10.5/21.0. Average score (-5.5)
Win/lose count 10.5/13.0. Average score (-4.9)
Win/lose count 7.5/15.0. Average score (-5.333333333333333)
Win/lose count 7.0/6.0. Average score (-4.428571428571429)
Win/lose count 11.5/17.0. Average score (-4.5625)
Win/lose count 11.5/10.0. Average score (-3.888888888888889)
Win/lose count 12.0/21.0. Average score (-4.4)
Final score: -4.4


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




> Here, we show that $Q^{\pi}$ follows the Bellman equation:
>
> 
> \begin{equation*}
 \begin{aligned}
Q^{\pi}(s, a) &= \mathbb{E}(\sum_{t \le T} \gamma^t r(s_t, a_t) | s_0 = s, a_0 = a, \cdots, a_t=\pi(s_t)) \\
            &= r(s, a) + \mathbb{E}(\sum_{1 \le t \le T} \gamma^t r(s_t, \pi(a_t)) | s_0 = s, a_0 = a)
\end{aligned}
\end{equation*}
>
> Then using the law of total expectations:
>
> 
>\begin{equation*}
\begin{aligned}
Q^{\pi}(s, a) &= r(s, a) + \gamma \sum_{s', a'} \mathbb{P}(s_1 = s', a_1 = a'| s_0 = s, a_0 = a)\mathbb{E}(\sum_{1 \le t \le T} \gamma^{t-1} r(s_t, \pi(s_t)) | s_1 = s', a_1 = a', \pi) \\
               &= E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{aligned}
\end{equation*}
> 
> Fo the second inequatily, we use the fact that $Q^* = \max_{\pi}Q^\pi$ and, with similar derivations:
>
>\begin{equation*}
\begin{aligned}
Q^*(s, a) &= r(s, a) + \gamma \sum_{s'} \max_{a'} \mathbb{P}(s_1 = s', a_1 = a'| s_0 = s, a_0 = a)\mathbb{E}(\sum_{1 \le t \le T} \gamma^{t-1} r(s_t, \pi^*(s_t)) | s_1 = s', a_1 = a', \pi^*) \\
               &= E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma \max_{a'}Q^*(s',a')]
\end{aligned}
\end{equation*}
>
> The learning problem is translated into finding the optimal policy $\pi^*$. Instead of looking directly for its value, we look for the optimal action-value function $Q^*$.
>
> Using the fact that the optimal Bellman operator admits a fixed point which is $Q^*$, we can deduce that
> $\mathcal{L}$ is a correct loss and represent the temporal difference between $Q_t$ and $Q_{t+1}$.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        if len(self.memory) > self.max_memory:
            self.memory.pop(0)

    def random_access(self):
        id_random = np.random.randint(len(self.memory))
        return self.memory[id_random]

***
The pipeline we will use for training is given below:

In [10]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [13]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s.reshape([1,s.shape[0],s.shape[1],s.shape[2]]))[0,:])

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ######## FILL IN
            s_, n_s_, a_, r_, game_over_  = self.memory.random_access()
            
            target_q[i] =self.model.predict(s_.reshape([1,s_.shape[0],s_.shape[1],s_.shape[2]]))[0]
            
            input_states[i] = s_
            
            if game_over_:
                ######## FILL IN
                target_q[i,a_] = r_
            else:
                ######## FILL IN
                q_est = self.model.predict(np.expand_dims(n_s_, 0))
                target_q[i, a_] = r_ + self.discount * np.max(q_est)
                
                
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)
        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        #Best model
        model = Sequential()
        model.add(Flatten(input_shape=(5,5,self.n_state,)))
        model.add(Dense(32,activation ='relu')) 
        model.add(Dense(4))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [14]:
epochs_train = 15
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 16)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Epoch 000/015 | Loss 0.0034 | Win/lose count 6.5/3.0 (3.5)
Epoch 001/015 | Loss 0.0027 | Win/lose count 2.5/3.0 (-0.5)
Epoch 002/015 | Loss 0.0297 | Win/lose count 7.5/8.0 (-0.5)
Epoch 003/015 | Loss 0.0068 | Win/lose count 4.0/3.0 (1.0)
Epoch 004/015 | Loss 0.0091 | Win/lose count 4.5/6.0 (-1.5)
Epoch 005/015 | Loss 0.0059 | Win/lose count 3.5/3.0 (0.5)
Epoch 006/015 | Loss 0.0210 | Win/lose count 7.5/2.0 (5.5)
Epoch 007/015 | Loss 0.0186 | Win/lose count 5.5/1.0 (4.5)
Epoch 008/015 | Loss 0.0034 | Win/lose count 6.5/1.0 (5.5)
Epoch 009/015 | Loss 0.0043 | Win/lose count 3.0/6.0 (-3.0)
Epoch 010/015 | Loss 0.0162 | Win/lose count 3.0/3.0 (0.0)
Epoch 011/015 | Loss 0.0029 | Win/lose count 4.5/4.0 (0.5)
Epoch 012/015 | Loss 0.0044 | Win/lose count 6.5/3.0 (3.5)
Epoch 013/015 | Loss 0.0040 | Win/lose count 4.5/2.0 (2.5)
Epoch 014/015 | Loss 0.0275 | Win/lose count 4.0/5.0 (-1.0)


***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [15]:
class DQN_CNN(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__( *args,**kwargs)
        
        model = Sequential()
        #Best model
        model.add(Conv2D(32,(2,2),input_shape=(5,5,self.n_state,),activation='relu'))
        model.add(Conv2D(64,(2,2),activation='relu'))
        model.add(Flatten())
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [16]:
epochs_train = 15
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=0.1, epsilon = 0.1, memory_size=2000, batch_size = 16)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/015 | Loss 0.0015 | Win/lose count 5.0/2.0 (3.0)
Epoch 001/015 | Loss 0.0090 | Win/lose count 6.0/3.0 (3.0)
Epoch 002/015 | Loss 0.0028 | Win/lose count 4.0/6.0 (-2.0)
Epoch 003/015 | Loss 0.0019 | Win/lose count 3.0/3.0 (0.0)
Epoch 004/015 | Loss 0.0015 | Win/lose count 2.5/5.0 (-2.5)
Epoch 005/015 | Loss 0.0070 | Win/lose count 2.0/0 (2.0)
Epoch 006/015 | Loss 0.0028 | Win/lose count 1.5/3.0 (-1.5)
Epoch 007/015 | Loss 0.0848 | Win/lose count 4.5/4.0 (0.5)
Epoch 008/015 | Loss 0.1051 | Win/lose count 5.5/2.0 (3.5)
Epoch 009/015 | Loss 0.0020 | Win/lose count 9.5/3.0 (6.5)
Epoch 010/015 | Loss 0.0030 | Win/lose count 9.0/3.0 (6.0)
Epoch 011/015 | Loss 0.0009 | Win/lose count 2.5/3.0 (-0.5)
Epoch 012/015 | Loss 0.0156 | Win/lose count 3.0/1.0 (2.0)
Epoch 013/015 | Loss 0.0021 | Win/lose count 14.0/1.0 (13.0)
Epoch 014/015 | Loss 0.0027 | Win/lose count 9.0/3.0 (6.0)


***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [36]:
env = Environment(grid_size=size, max_time=T,temperature=0.9)

agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')

epochs_test = 20

print('Testing of the FC Based Agent')
test(agent_fc,env,epochs_test,prefix='fc_test')

print('\nTesting CNN Based Agent')
test(agent_cnn,env,epochs_test,prefix='cnn_test')

Testing of the FC Based Agent
Win/lose count 4.5/0. Average score (4.5)
Win/lose count 15.0/0. Average score (9.75)
Win/lose count 6.5/0. Average score (8.666666666666666)
Win/lose count 21.0/1.0. Average score (11.5)
Win/lose count 2.5/0. Average score (9.7)
Win/lose count 7.5/0. Average score (9.333333333333334)
Win/lose count 6.0/0. Average score (8.857142857142858)
Win/lose count 22.5/1.0. Average score (10.4375)
Win/lose count 3.5/0. Average score (9.666666666666666)
Win/lose count 2.5/0. Average score (8.95)
Win/lose count 11.5/0. Average score (9.181818181818182)
Win/lose count 2.0/0. Average score (8.583333333333334)
Win/lose count 9.5/0. Average score (8.653846153846153)
Win/lose count 3.0/0. Average score (8.25)
Win/lose count 6.0/0. Average score (8.1)
Win/lose count 13.0/0. Average score (8.40625)
Win/lose count 10.5/1.0. Average score (8.470588235294118)
Win/lose count 12.5/0. Average score (8.694444444444445)
Win/lose count 20.0/1.0. Average score (9.236842105263158)
Win/

In [37]:
HTML(display_videos('cnn_test10.mp4'))

In [38]:
HTML(display_videos('fc_test10.mp4'))

Temperature increases the density of the reward pixel. Therefore it increase the score. It's also observed that the rat does not explore the board and gets stuck in between 2 positions and keep oscillating there.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [17]:
def train_explore(agent,env,epoch, epsilon_start=0.7, epsilon_end=0.1, epsilon_decay=60000, prefix=''):
    # Number of won games
    score = 0
    loss = 0
    steps_done = 0
    
    agent.set_epsilon(epsilon_start)
    
    for e in range(epoch):
       
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
            
            eps_threshold = epsilon_start + (epsilon_end - epsilon_start) * steps_done / epsilon_decay
        
            agent.set_epsilon(eps_threshold)
            steps_done += 1

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(Environment):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        super(EnvironmentExploring, self).__init__(grid_size, max_time, temperature)
        
        self.malus_position = np.zeros_like(self.position)
    
    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        
        # Adding explored cases
        self.malus_position[self.x, self.y] += 0.1
        
        if action == 0:  # Right
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:  # Left
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:  # Up
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:  # Down
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: This action is not recognized')
            

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        if train:
            reward -= self.malus_position[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1),
                                self.malus_position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.malus_position = np.zeros_like(self.position)
        
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1),
                                self.malus_position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [18]:
# Training

env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.001, epsilon = 0.9, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, 51, epsilon_start=0.7, epsilon_end=0.1, 
              epsilon_decay=60000, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore50.mp4'))

Epoch 000/051 | Loss 0.0353 | Win/lose count 5.5/64.90000000000003 (-59.400000000000034)
Epoch 001/051 | Loss 0.0619 | Win/lose count 11.5/37.60000000000004 (-26.100000000000037)
Epoch 002/051 | Loss 0.0421 | Win/lose count 8.5/49.600000000000016 (-41.100000000000016)
Epoch 003/051 | Loss 0.0380 | Win/lose count 11.0/43.90000000000003 (-32.90000000000003)
Epoch 004/051 | Loss 0.0362 | Win/lose count 12.0/45.20000000000004 (-33.20000000000004)
Epoch 005/051 | Loss 0.0595 | Win/lose count 6.5/81.99999999999997 (-75.49999999999997)
Epoch 006/051 | Loss 0.0638 | Win/lose count 14.0/45.500000000000036 (-31.500000000000036)
Epoch 007/051 | Loss 0.0249 | Win/lose count 10.0/54.60000000000004 (-44.60000000000004)
Epoch 008/051 | Loss 0.0528 | Win/lose count 10.5/54.400000000000006 (-43.900000000000006)
Epoch 009/051 | Loss 0.0387 | Win/lose count 16.5/39.30000000000001 (-22.80000000000001)
Epoch 010/051 | Loss 0.0463 | Win/lose count 11.5/122.10000000000001 (-110.60000000000001)
Epoch 011/051 

In [20]:
# Evaluation
test(agent,env,21,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 9.5/12.0. Average score (-2.5)
Win/lose count 12.0/11.0. Average score (-0.75)
Win/lose count 10.5/5.0. Average score (1.3333333333333333)
Win/lose count 11.0/7.0. Average score (2.0)
Win/lose count 9.5/12.0. Average score (1.1)
Win/lose count 10.0/5.0. Average score (1.75)
Win/lose count 6.0/6.0. Average score (1.5)
Win/lose count 10.0/6.0. Average score (1.8125)
Win/lose count 3.0/1.0. Average score (1.8333333333333333)
Win/lose count 11.0/8.0. Average score (1.95)
Win/lose count 12.0/9.0. Average score (2.0454545454545454)
Win/lose count 6.0/6.0. Average score (1.875)
Win/lose count 13.0/7.0. Average score (2.1923076923076925)
Win/lose count 13.5/8.0. Average score (2.4285714285714284)
Win/lose count 10.0/5.0. Average score (2.6)
Win/lose count 10.0/6.0. Average score (2.6875)
Win/lose count 12.0/9.0. Average score (2.7058823529411766)
Win/lose count 9.5/10.0. Average score (2.5277777777777777)
Win/lose count 5.5/5.0. Average score (2.4210526315789473)
Win/lose count 

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

In [21]:
class Mimic_Agent(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(Mimic_Agent, self).__init__(epsilon = epsilon)
        self.discount = 0.99
        self.grid_size = grid_size
        self.n_state = n_state
        self.memory = Memory(memory_size)
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s.reshape([1,s.shape[0],s.shape[1],s.shape[2]]))[0,:])

    def reinforce(self, s_, a_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_[:,:,1:], a_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            s_, a_  = self.memory.random_access()
            
            target_q[i, a_] = 5
            input_states[i] = s_
        
        l = self.model.train_on_batch(input_states, target_q)
        
        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model
        
class CNN_Mimic_Agent(Mimic_Agent):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(CNN_Mimic_Agent, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(50,(2,2),input_shape=(5,5,self.n_state,),activation='relu'))
        model.add(Flatten())
        model.add(Dense(4))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        
# Training the Mimic Agent
def train_mimic_agent(ref, agent,env,epoch,epsilon=0.1, decay=0.99, prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        
        agent.set_epsilon(epsilon)

        while not game_over:
            # The agent performs an action
            action = ref.act(state, train=False)

            
            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
                
            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state,  action)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

In [22]:
epochs_train = 15
pro_agent = agent
mimic_agent = CNN_Mimic_Agent(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=2)
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
train_mimic_agent(pro_agent,mimic_agent, env, epochs_train, prefix='train_cnn_mimick_agent')

Epoch 000/015 | Loss 2.6434 | Win/lose count 12.5/70.70000000000002 (-58.20000000000002)
Epoch 001/015 | Loss 3.0209 | Win/lose count 11.0/43.80000000000002 (-32.80000000000002)
Epoch 002/015 | Loss 2.5984 | Win/lose count 1.0/399.50000000000006 (-398.50000000000006)
Epoch 003/015 | Loss 2.5309 | Win/lose count 12.5/48.000000000000014 (-35.500000000000014)
Epoch 004/015 | Loss 2.7409 | Win/lose count 9.0/46.400000000000034 (-37.400000000000034)
Epoch 005/015 | Loss 2.6728 | Win/lose count 1.5/849.8999999999993 (-848.3999999999993)
Epoch 006/015 | Loss 2.5566 | Win/lose count 9.0/69.20000000000005 (-60.200000000000045)
Epoch 007/015 | Loss 1.9380 | Win/lose count 9.5/60.40000000000004 (-50.90000000000004)
Epoch 008/015 | Loss 2.9507 | Win/lose count 7.5/42.90000000000002 (-35.40000000000002)
Epoch 009/015 | Loss 2.5872 | Win/lose count 6.5/70.10000000000001 (-63.60000000000001)
Epoch 010/015 | Loss 3.0017 | Win/lose count 7.0/62.20000000000002 (-55.20000000000002)
Epoch 011/015 | Loss 2

In [23]:
# Evaluation
env_test = Environment(grid_size=size, max_time=T,temperature=0.3)
test(mimic_agent,env_test,epochs_test,prefix='test_cnn_mimick_agent')

Win/lose count 2.5/3.0. Average score (-0.5)
Win/lose count 0.5/0. Average score (0.0)
Win/lose count 2.5/1.0. Average score (0.5)
Win/lose count 1.0/1.0. Average score (0.375)
Win/lose count 0.5/0. Average score (0.4)
Win/lose count 0/3.0. Average score (-0.16666666666666666)
Win/lose count 0/0. Average score (-0.14285714285714285)
Win/lose count 1.0/8.0. Average score (-1.0)
Win/lose count 0.5/0. Average score (-0.8333333333333334)
Win/lose count 2.0/5.0. Average score (-1.05)
Final score: -1.05


In [26]:
HTML(display_videos('test_cnn_mimick_agent0.mp4'))

> Performance of the agent drops significantly but training becomes faster.

***